In [ ]:
from npc_base import GeneralNPC_AI  # Import the base AI model
from torch_geometric.data import Data

In [ ]:
states = ("Wander","Still", "Interact","Charge", "Attack")

In [1]:
class BasicEnemyAI(nn.Module):
    """AI model to predict the best state-action pair."""
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BasicEnemyAI, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        q_values = self.fc2(x)
        return q_values

SyntaxError: incomplete input (2393897256.py, line 1)

In [ ]:
class BasicEnemy(NPC):
    def __init__(self, map_size, learning_rate=0.01):
        super().__init__(health=1, input_dim=4, hidden_dim=8, num_actions=4, map_size=map_size)
        self.position = (random.randint(0, map_size - 1), random.randint(0, map_size - 1))
        self.ai_model = BasicEnemyAI(input_dim=4, hidden_dim=16, output_dim=4)  # 4 states/actions
        self.optimizer = optim.Adam(self.ai_model.parameters(), lr=learning_rate)
        self.criterion = nn.MSELoss()
        self.state = "Still"
        self.action_space = ["Wander", "Still", "Charge", "Attack"]

    def get_state(self, player_position):
        """Convert the current state into a tensor input."""
        px, py = player_position
        nx, ny = self.position
        return torch.tensor([px - nx, py - ny, nx, ny], dtype=torch.float32).unsqueeze(0)

    def choose_action(self, state, epsilon=0.1):
        """Epsilon-greedy action selection."""
        if random.random() < epsilon:  # Explore with probability epsilon
            return random.choice(range(4))
        else:  # Exploit based on learned Q-values
            with torch.no_grad():
                q_values = self.ai_model(state)
                return torch.argmax(q_values).item()

    def update_q_values(self, state, action, reward, next_state, gamma=0.99):
        """Update Q-values using Bellman Equation."""
        self.optimizer.zero_grad()
        q_values = self.ai_model(state)
        next_q_values = self.ai_model(next_state).detach()
        target = q_values.clone()
        target[0, action] = reward + gamma * torch.max(next_q_values)
        loss = self.criterion(q_values, target)
        loss.backward()
        self.optimizer.step()

    def move(self, action):
        """Execute the action: Wander, Charge, Attack, or Stay Still."""
        if self.action_space[action] == "Wander":
            self.move_randomly()
            print(f"BasicEnemy wanders to {self.position}")
        elif self.action_space[action] == "Charge":
            self.move_toward_player(player_position)
            print(f"BasicEnemy charges toward {player_position}")
        elif self.action_space[action] == "Attack":
            print("BasicEnemy attacks the player with a melee attack!")
        elif self.action_space[action] == "Still":
            print("BasicEnemy stays still.")

    def move_randomly(self):
        """Helper function for wandering."""
        x, y = self.position
        move_options = [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]
        valid_moves = [(nx, ny) for nx, ny in move_options 
                       if 0 <= nx < self.ai_model.map_size and 0 <= ny < self.ai_model.map_size]
        if valid_moves:
            self.position = random.choice(valid_moves)

    def move_toward_player(self, player_position):
        """Move one step closer to the player."""
        px, py = player_position
        nx, ny = self.position
        if px > nx:
            self.position = (nx + 1, ny)
        elif px < nx:
            self.position = (nx - 1, ny)
        elif py > ny:
            self.position = (nx, ny + 1)
        elif py < ny:
            self.position = (nx, ny - 1)

In [ ]:
map_size = 10
basic_enemy = BasicEnemy(map_size)
player_position = (5, 5)

# Simulate behavior with AI decision-making
for _ in range(10):
    state = basic_enemy.get_state(player_position)
    action = basic_enemy.choose_action(state)
    basic_enemy.move(action)